# Working with Data - Le Xu

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog 

In [1]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [2]:
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [3]:
data.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11224,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)"
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,NaN,NaN,NaN,1-1-1173130914,NaN,NaN,NaN,NaN,NaN,NaN
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,Street,11234,1157 EAST 57 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)"
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11218,722 EAST 4 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)"
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NYC Street Address,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
data1 = data

In [144]:
data1.info() # to take general look of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10187766 entries, 0 to 10187765
Data columns (total 54 columns):
Unique Key                        int64
Created Date                      object
Closed Date                       object
Agency                            object
Agency Name                       object
Complaint Type                    object
Descriptor                        object
Location Type                     object
Incident Zip                      object
Incident Address                  object
Street Name                       object
Cross Street 1                    object
Cross Street 2                    object
Intersection Street 1             object
Intersection Street 2             object
Address Type                      object
City                              object
Landmark                          object
Facility Type                     object
Status                            object
Due Date                          object
Resolution Description   

In [145]:
# Check if all Boroughs and Community Districts are represented in the Data 

In [146]:
data1.Borough.unique()  #Five boroughs and one unspecified 

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [147]:
data1['Community Board'].unique()  

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

# Q: How many unique values do we have? 

In [148]:
len(data1['Community Board'].unique())

77

## A: We have 77 unique values, which is not correct. We need to have 59 community districts. Some of them are unspecified, missing. Some might be invalid entries. 


In [149]:
data_new = data1.loc[:,["Agency Name","Incident Zip","Community Board","Borough","Latitude","Longitude"]] #narrow down 

## Check for duplicates? Are these plausible?

In [150]:
data_new = data_new.drop_duplicates()

In [153]:
len(data_new['Community Board'])

2955956

In [154]:
len(data_new['Community Board'].unique())

77

## By narrowed down to six columns, the drop_duplicateds showed me 2,955,956 records, instead of original 10,187,766 records. Since I included latitude, longtitude, angecy name, so I can make some prediction that some complaints were made by same people/location. But the numbers community Board stays at 77. 
---

# Let's deal with the Missing values? 

In [155]:
data1.isnull().sum()

Unique Key                               0
Created Date                             0
Closed Date                         450879
Agency                                   0
Agency Name                              0
Complaint Type                           0
Descriptor                           31057
Location Type                      3067596
Incident Zip                        794159
Incident Address                   2197706
Street Name                        2198559
Cross Street 1                     2375943
Cross Street 2                     2425793
Intersection Street 1              8345288
Intersection Street 2              8346302
Address Type                        490299
City                                788712
Landmark                          10180456
Facility Type                      8579485
Status                                   0
Due Date                           7120286
Resolution Description                   0
Resolution Action Updated Date      250934
Community B

## Replacing Missing Values:
### to fill missings in Resolution Descriptor with Complaint Type

In [156]:
data1["Resolution Description"].fillna(value = data1["Complaint Type"], inplace = True )

In [157]:
data1["Resolution Description"].isnull().sum()

0

## Find the unplausible Community Districts 

### From the resource  http://nyc.pediacities.com/Community_Districts, we can see there are total 12 Manhattan Dist, and 18 in Brooklyn, 14 in Queens, 12 in Bronx, 3 in Staten Island.  

In [158]:
# Drop the marked districts
data1["mask"]= data1['Community Board'].map(lambda x: (x[:2]))


In [159]:
data1["mask"].value_counts()
# We have to clean the data

0     1109446
Un     955186
12     816211
01     691859
03     681232
05     661441
07     640140
09     562097
04     553635
02     550604
08     506059
10     472793
11     459292
06     408763
14     291190
13     236213
17     191720
18     156100
15     146129
16      85467
83       3072
80       2363
81       1420
64        904
56        834
55        767
82        725
95        531
27        469
28        433
26        397
84        274
Name: mask, dtype: int64

In [390]:
#make the string into numbers
data1["mask"] = pd.to_numeric(data1["mask"], errors='coerce')

In [391]:
# district number between 1(min)-18(max)
data2 = data1[data1["mask"]<19]
data2 = data2[data2["mask"]>0]

In [392]:
len(data2['Community Board'].unique())

59

                      Finally, we are able to get 59 unique Comunity Board.
---

## Generate an indicator which ranks the Community District by complaint numbers on the Community district level

In [393]:
# Safe reduced data frame (Community District level)
data2_g = data2["Unique Key"].groupby(data2["Community Board"]).count()
data2_g = pd.DataFrame(data2_g)
data2_g["Rank by Complaints"] = data2_g.rank(ascending=False).sort()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [394]:
data2_sorted = data2_g.sort("Rank by Complaints").reset_index()
data2_sorted.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Community Board,Unique Key,Rank by Complaints
0,12 MANHATTAN,268053,1.0
1,12 QUEENS,229383,2.0
2,03 BROOKLYN,197306,3.0
3,07 QUEENS,195961,4.0
4,17 BROOKLYN,191720,5.0


## I ranked the Community districts by the Unique Key of each record. It shows the Manhattan 12 district has the most complaints. 
---

# 4. Think about the interpretation of the results?

## Population by Community District

In [395]:
# Population by Community District
df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")

In [396]:
# Check variables in file
df_pop.head(10)

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04
5,3603705,BX06,172247,39405.79222,44.72,14.24,14.89,12.38,28.21,59.53,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04
6,3603706,BX07,135893,86487.07792,48.48,10.58,14.97,15.32,29.10,62.49,...,2590,4653,21.49,31.83,24.74,0.00,9.34,12.60,4.51,8.09
7,3603707,BX05,132850,87974.34860,45.75,13.10,16.34,11.96,29.84,59.43,...,1927,5345,16.10,39.78,21.56,0.11,8.19,14.26,3.78,10.48
8,3603708,BX04,141467,71270.88219,45.64,12.28,12.41,13.10,42.97,47.55,...,1762,6444,17.47,37.11,23.89,0.00,6.03,15.50,3.33,12.17
9,3603709,BX09,190126,42752.50690,47.59,11.15,15.22,12.54,33.62,61.02,...,3061,7466,22.34,29.50,24.34,0.00,10.17,13.65,3.97,9.68


In [397]:
df_pop.columns

Index(['FIPS', 'cd_id', 'Total Population',
       'Population Density (per sq. mile)', '% Total Population: Male',
       '% Total Population: 18 to 24 Years',
       '% Total Population: 25 to 34 Years',
       '% Total Population: 35 to 44 Years',
       '% Population 5 Years And Over: Speak Only English',
       '% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       '% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       '% Employed Civilian Population 16 Years And Over: Service occupations',
       '% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       '% Employed Civ

In [398]:
# How many community districts are in file? 
df_pop.cd_id.unique()

array(['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05',
       'BX04', 'BX09', 'BX01', 'BX02', 'MN12', 'MN09', 'MN10', 'MN11111',
       'MN08', 'MN07', 'MN05', 'MN04', 'MN06', 'MN03', 'MN01', 'MN02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06',
       'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10',
       'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09',
       'QN12', 'QN10', 'QN14'], dtype=object)

## Q: Who might be more likely to call 311
    A: People who live in crowded areas

In [399]:
# Manipulate data to get some information on demographics by Community District. 

In [400]:
len(df_pop.cd_id.unique())

59

In [401]:
df_pop.groupby(df_pop.cd_id).sum()

,FIPS,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,"% Population 5 Years And Over: Spanish or Spanish Creole: Speak English ""very Well""",...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
cd_id,,,,,,,,,,,,,,,,,,,,,
BK01,3604001,154713,37671.510580,49.93,8.52,29.88,13.70,72.48,15.53,9.81,...,1823,2894,52.80,15.35,22.83,0.00,3.47,5.55,2.14,3.40
BK02,3604004,139070,44719.888150,45.99,9.57,22.68,17.05,57.49,32.56,21.65,...,1196,1830,61.18,12.70,19.14,0.25,2.56,4.18,1.65,2.53
BK03,3604003,133235,58004.868160,47.55,12.75,19.43,11.55,77.10,17.30,10.83,...,1401,3220,38.00,25.77,23.45,0.34,4.57,7.87,2.39,5.48
BK04,3604002,137368,54279.761230,48.98,14.57,24.00,13.76,78.17,9.87,6.30,...,2500,4236,32.48,26.22,20.84,0.78,9.73,9.94,3.69,6.25
BK05,3604008,155863,26394.764690,45.49,11.18,15.29,12.60,25.03,37.69,16.50,...,2532,4407,22.95,31.85,26.95,0.00,6.98,11.27,4.11,7.16
BK06,3604005,125062,31218.503280,48.71,4.79,22.77,21.13,69.33,6.48,3.81,...,660,1121,72.16,6.67,16.34,0.00,2.30,2.53,0.94,1.59
BK07,3604012,152685,36736.915040,49.70,9.38,19.44,16.10,43.33,5.24,3.01,...,4255,3886,28.47,33.45,19.57,0.05,7.42,11.04,5.77,5.27
BK08,3604006,126747,57010.543560,46.57,10.12,19.57,16.61,80.23,5.63,3.61,...,939,3774,39.90,26.36,21.15,0.00,4.91,7.68,1.53,6.15
BK09,3604011,109337,64006.179970,45.55,11.44,20.14,12.49,50.85,13.08,5.85,...,944,2414,41.95,24.92,20.82,0.00,5.68,6.63,1.86,4.76


In [402]:
# Save data frame
df_pop_g=df_pop.groupby(df_pop.cd_id).sum()

In [430]:
df_pop_sort = df_pop_g.sort('Total Population',ascending=False).reset_index()
df_pop_sort["Community Board"] = df_pop_sort["cd_id"]

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [449]:
df_pop_sort = df_pop_sort.iloc[:,(2,-1)]

In [450]:
df_pop_sort.head()

,Total Population,Community Board
0,256742,QN07
1,251002,QN12
2,224365,MN12
3,221898,MN08
4,210957,BK18


In [451]:
data2_sorted.head()

,Community Board,Unique Key,Rank by Complaints
0,MA12,268053,1.0
1,QN12,229383,2.0
2,BK03,197306,3.0
3,QN07,195961,4.0
4,BK17,191720,5.0


## The first dataset showed the most top 5 populated districts, and the second showed the top 5 ranked districts by complaints. 
## By comparing those two, QN07, MN12, QN12 appeared on both top 5 lists, as a result, we can assume that the more population in the area, the more 311 complaints could the area has.

---

### Infrastructure by Community District

In [405]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [406]:
# Check variables in file
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [407]:
df_infr["Qualifying Name"]

0     NYC-Bronx Community District 8--Riverdale, New...
1     NYC-Bronx Community District 12--Wakefield, Ne...
2     NYC-Bronx Community District 10--Co-op City, N...
3     NYC-Bronx Community District 11--Pelham Parkwa...
4     NYC-Bronx Community District 3 & 6--Belmont, N...
5     NYC-Bronx Community District 3 & 6--Belmont, N...
6     NYC-Bronx Community District 7--Bedford Park, ...
7     NYC-Bronx Community District 5--Morris Heights...
8     NYC-Bronx Community District 4--Concourse, New...
9     NYC-Bronx Community District 9--Castle Hill, N...
10    NYC-Bronx Community District 1 & 2--Hunts Poin...
11    NYC-Bronx Community District 1 & 2--Hunts Poin...
12    NYC-Manhattan Community District 12--Washingto...
13    NYC-Manhattan Community District 9--Hamilton H...
14    NYC-Manhattan Community District 10--Central H...
15    NYC-Manhattan Community District 11--East Harl...
16    NYC-Manhattan Community District 8--Upper East...
17    NYC-Manhattan Community District 7--Upper 

In [408]:
# How many community districts are in file? 
len(df_infr["Qualifying Name"])

59

In [409]:
len(df_infr["Qualifying Name"].unique())

55

     The difference of the community districts here is becuase there are 8 districts were grouped into 4 districts.

## Aggregate the mobile subscription data

In [410]:
df_infr.columns

Index(['FIPS', 'Geographic Identifier', 'Qualifying Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Househol

In [411]:
df_usage = df_infr[["Qualifying Name","Households: With An Internet Subscription",'Households: Without An Internet Subscription',
         'Households: No Computer']]

In [412]:
df_usage["Without Internet"] = df_usage["Households: Without An Internet Subscription"]+df_usage["Households: No Computer"]

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [413]:
df_usage = df_usage.iloc[:,(0,1,4)]


In [414]:
df_usage.head()

,Qualifying Name,Households: With An Internet Subscription,Without Internet
0,"NYC-Bronx Community District 8--Riverdale, New...",31795,10688
1,"NYC-Bronx Community District 12--Wakefield, Ne...",32243,13217
2,"NYC-Bronx Community District 10--Co-op City, N...",32729,14559
3,NYC-Bronx Community District 11--Pelham Parkwa...,32003,13522
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",35503,24060


## Aggregate internet type by high and low connections

In [415]:
df_connection = df_infr[["Qualifying Name",'Households: With A Broadband Internet Subscription',
         'Households: With Dial-Up Internet Subscription Alone']]
df_connection.head()

,Qualifying Name,Households: With A Broadband Internet Subscription,Households: With Dial-Up Internet Subscription Alone
0,"NYC-Bronx Community District 8--Riverdale, New...",30943,404
1,"NYC-Bronx Community District 12--Wakefield, Ne...",31435,178
2,"NYC-Bronx Community District 10--Co-op City, N...",32333,158
3,NYC-Bronx Community District 11--Pelham Parkwa...,31278,122
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",33408,88


In [438]:
df_connection.columns = ["Community Board","High connection","Low connection"]
df_connection.head()

,Community Board,High connection,Low connection
0,"NYC-Bronx Community District 8--Riverdale, New...",30943,404
1,"NYC-Bronx Community District 12--Wakefield, Ne...",31435,178
2,"NYC-Bronx Community District 10--Co-op City, N...",32333,158
3,NYC-Bronx Community District 11--Pelham Parkwa...,31278,122
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",33408,88


In [417]:
# Save data frame 

# 5. Inform your results by linking datasets
   I understand it will be better to use re, however, I am not familiar with it yet. I will learn more and use it in the future.

In [418]:
new = []
for i in data2_sorted["Community Board"]:
    match = re.search("(.+) (.+)", i )
    #print (match.group(1))
    res = [];
    if match.group(2) == 'MANHATTAN':
        res.append("MA")
        res.append(match.group(1))
    elif match.group(2) == 'BROOKLYN':
        res.append("BK")
        res.append(match.group(1))
    elif match.group(2) == 'BRONX':
        res.append("BX")
        res.append(match.group(1))
    elif match.group(2) == 'ISLAND':
        res.append("SI")
        res.append(match.group(1)[:2])
    elif match.group(2) == 'QUEENS':
        res.append("QN")
        res.append(match.group(1))
    new.append(''.join(res))
print (new)

['MA12', 'QN12', 'BK03', 'QN07', 'BK17', 'BK05', 'BK14', 'BX07', 'BK01', 'SI01', 'BX04', 'QN01', 'QN05', 'BX12', 'QN13', 'BX05', 'BK18', 'MA07', 'QN09', 'BK12', 'MA03', 'MA08', 'BX09', 'BK15', 'MA05', 'BK08', 'BK11', 'BK04', 'MA10', 'SI03', 'BK09', 'MA02', 'MA04', 'QN10', 'MA09', 'QN08', 'BX11', 'QN03', 'SI02', 'BK02', 'QN02', 'BK06', 'BK10', 'BK07', 'MA06', 'QN14', 'QN11', 'BX06', 'QN04', 'BX10', 'MA11', 'BX08', 'QN06', 'BK16', 'MA01', 'BX03', 'BX01', 'BK13', 'BX02']


In [419]:
data2_sorted["Community Board"] = new

In [420]:
# Harmonize identifier of dataframe 1
data2_sorted.head()

,Community Board,Unique Key,Rank by Complaints
0,MA12,268053,1.0
1,QN12,229383,2.0
2,BK03,197306,3.0
3,QN07,195961,4.0
4,BK17,191720,5.0


In [421]:
# Harmonize identifier of dataframe 

df_infr["Qualifying Name"].head()

0    NYC-Bronx Community District 8--Riverdale, New...
1    NYC-Bronx Community District 12--Wakefield, Ne...
2    NYC-Bronx Community District 10--Co-op City, N...
3    NYC-Bronx Community District 11--Pelham Parkwa...
4    NYC-Bronx Community District 3 & 6--Belmont, N...
Name: Qualifying Name, dtype: object

In [423]:
new = []
for i in df_infr["Qualifying Name"]:
    cur = i.split(' ') 
    res = [];
    if cur[0] == 'NYC-Manhattan':
        res.append("MA")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Brooklyn':
        res.append("BK")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Bronx':
        res.append("BX")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Staten':
        res.append("SI")
        num = cur[4].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Queens':
        res.append("QN")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    new.append(''.join(res))
print (new)

['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX03', 'BX07', 'BX05', 'BX04', 'BX09', 'BX01', 'BX01', 'MA12', 'MA09', 'MA10', 'MA11', 'MA08', 'MA07', 'MA04', 'MA04', 'MA06', 'MA03', 'MA01', 'MA01', 'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07', 'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14']


In [427]:
new = ['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05', 'BX04', 'BX09', 'BX01', 'BX02',
       'MA12', 'MA09', 'MA10', 'MA11', 'MA08', 'MA07', 'MA04', 'MA05', 'MA06', 'MA03', 'MA01', 'MA02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18',
       'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14']

In [440]:
df_connection["Community Board"] = new
df_connection.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Community Board,High connection,Low connection
0,BX08,30943,404
1,BX12,31435,178
2,BX10,32333,158
3,BX11,31278,122
4,BX03,33408,88


In [ ]:
# Harmonize identifier of dataframe 3

In [454]:
# Link the 3 dataframes
united = pd.merge(df_connection, data2_sorted, on = "Community Board",how ="inner")
united = pd.merge(united, df_pop_sort, on = "Community Board" )
united.sort("High connection")

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Community Board,High connection,Low connection,Unique Key,Rank by Complaints,Total Population
21,BK16,24376,64,85467,54.0,123772
7,BX05,25772,54,161078,16.0,132850
32,BK13,27559,191,72362,58.0,110727
28,BK12,27609,268,150320,20.0,170011
46,QN14,27803,178,103396,46.0,107944
8,BX04,27971,316,181953,11.0,141467
25,BK09,28023,334,135833,31.0,109337
45,QN10,29698,338,128116,34.0,134827
0,BX08,30943,404,92733,52.0,106737
10,BX01,30958,0,74631,57.0,167147


# Are the demographics and infrastructure different in Community Districts that show more complaints than others?


    It seems like the higher the internet connection, the more complaints. Maybe rich people likes to complain more or bigger fammilies likes to complain more. Also the more crowded the area, the more complaints.